# 1. Ziele von Data Cleaning

In diesem Notebook bereinige ich den ursprünglichen Smart-Meter-Datensatz, damit er für die weitere Analyse, Anomalieerkennung und das Forecasting zuverlässig genutzt werden kann.

Die wichtigsten Schritte sind:

- Laden der Rohdaten
- Umwandeln von Datum & Uhrzeit in einen eindeutigen Zeitstempel
- Konvertieren der Messspalten in numerische Werte
- Entfernen fehlender oder fehlerhafter Daten
- Prüfen der Struktur des Datensatzes
- Speichern der bereinigten Version

Dies bildet die Grundlage für alle weiteren Schritte (EDA, Anomalieerkennung, Dashboard, Forecasting).

### 1. Bibliotheken importieren und Pfade definieren

In [114]:
import pandas as pd
import numpy as np
import os
pfad_original="C:\\Users\\NataliaArchipenko\\Desktop\\Python\\Anomaly_Detection_Energieverbrauch\\Data\\original\\household_power_consumption.txt"
pfad_bearbeitet="C:\\Users\\NataliaArchipenko\\Desktop\\Python\\Anomaly_Detection_Energieverbrauch\\Data\\bearbeitet\\household_power_clean.csv"

### 2. Einlesen der originalen Daten

Die Originaldaten sind semikolon-separiert und enthalten fehlende Werte, die als ? markiert sind. Beim Einlesen werden diese direkt als NaN erkannt.

In [115]:
df_orig = pd.read_csv(
    pfad_original,
    sep=";",                 # Semicolon als Trennzeichen
    decimal=".",             # Dezimalpunkte
    na_values=["?"],         # Fehlerwerte im Datensatz
    low_memory=False         # bessere Speicherverwaltung
)
print(f'Anzahl Zeilen: {df_orig.shape[0]}, Anzahl Spalten: {df_orig.shape[1]}')

Anzahl Zeilen: 2075259, Anzahl Spalten: 9


In [116]:
df_orig.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075259 entries, 0 to 2075258
Data columns (total 9 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Date                   object 
 1   Time                   object 
 2   Global_active_power    float64
 3   Global_reactive_power  float64
 4   Voltage                float64
 5   Global_intensity       float64
 6   Sub_metering_1         float64
 7   Sub_metering_2         float64
 8   Sub_metering_3         float64
dtypes: float64(7), object(2)
memory usage: 142.5+ MB


In [117]:
df_orig.head(2)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
0,16/12/2006,17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
1,16/12/2006,17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0


In [118]:
df_orig.tail(2)

,Date,Time,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
2075257,26/11/2010,21:01:00,0.934,0.0,239.70,3.8,0.0,0.0,0.0
2075258,26/11/2010,21:02:00,0.932,0.0,239.55,3.8,0.0,0.0,0.0


In [119]:
df = df_orig.copy() #Originale Dataframe kopieren, um weiter zu bearbeiten

### 3. Konvertieren der Zeitinformation (Date + Time → Datetime)

Der Datensatz enthält Datum und Uhrzeit in zwei Spalten.
Für eine korrekte Zeitreihenanalyse ist jedoch ein eindeutiger Zeitstempel (Datetime) erforderlich.

Vorteile eines kombinierten Datetime-Feldes:
- eindeutiger, sortierbarer Zeitindex
- vereinfachtes Resampling (z. B. auf Stunden)
- Rolling-Windows für Glättung und Anomalieerkennung
- einfaches Feature Engineering (Stunde, Wochentag, Monat, Saison)
- ML-Modelle wie Prophet benötigen explizit eine einheitliche Timestamp-Spalte

Darum werden Date und Time zu einer Datetime-Spalte zusammengeführt und als Index gesetzt.

In [120]:
df["Datetime"] = pd.to_datetime(
    df["Date"] + " " + df["Time"],
    format="%d/%m/%Y %H:%M:%S"
)

In [121]:
df = df.set_index("Datetime")

In [122]:
df = df.drop(columns=["Date", "Time"]) # Löschen der Spalten Date und Time

### 4. Überprüfung & Bereinigung der Datentypen

Die Messspalten sind erwartungsgemäß numerisch.

Durch errors="coerce" werden fehlerhafte Werte automatisch in NaN umgewandelt.

In [123]:
df = df.apply(pd.to_numeric, errors="coerce")


In [124]:
df.dtypes

Global_active_power      float64
Global_reactive_power    float64
Voltage                  float64
Global_intensity         float64
Sub_metering_1           float64
Sub_metering_2           float64
Sub_metering_3           float64
dtype: object

In [125]:
df.head(3)

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
Datetime,,,,,,,
2006-12-16 17:24:00,4.216,0.418,234.84,18.4,0.0,1.0,17.0
2006-12-16 17:25:00,5.360,0.436,233.63,23.0,0.0,1.0,16.0
2006-12-16 17:26:00,5.374,0.498,233.29,23.0,0.0,2.0,17.0


### 5. Behandlung fehlender Werte

Im Datensatz fehlen ca. 1,25 % der Messwerte. Da:

- der Datensatz sehr groß ist
- die fehlenden Werte über alle Spalten verteilt auftreten
- ein Löschen keine Verzerrung erzeugt
- die Zeitreihe weiterhin vollständig bleibt
- entscheide ich mich, diese Zeilen komplett zu entfernen (***dropna()***).

Dies ist die sauberste Lösung, ohne die Datenqualität zu beeinträchtigen.

In [126]:
df.isna().sum()

Global_active_power      25979
Global_reactive_power    25979
Voltage                  25979
Global_intensity         25979
Sub_metering_1           25979
Sub_metering_2           25979
Sub_metering_3           25979
dtype: int64

In [127]:
df.isna().mean()  # Anteil in %

Global_active_power      0.012518
Global_reactive_power    0.012518
Voltage                  0.012518
Global_intensity         0.012518
Sub_metering_1           0.012518
Sub_metering_2           0.012518
Sub_metering_3           0.012518
dtype: float64

In [128]:
df = df.dropna() 

In [129]:
df.isna().sum()# Übeprüfen, ob alle Nan-Werte gelöscht wurden


Global_active_power      0
Global_reactive_power    0
Voltage                  0
Global_intensity         0
Sub_metering_1           0
Sub_metering_2           0
Sub_metering_3           0
dtype: int64

### 6. Entfernen potenziell unplausibler Werte

Falls Messfehler auftreten (z. B. negative Spannungen oder Leistungswerte), werden diese zu NaN gesetzt und ebenfalls entfernt. In diesem Datensatz treten solche Fälle sehr selten auf.

In [131]:
# Negative Werte sind bei Stromverbrauch unplausibel
numeric_cols = df.columns
df[numeric_cols] = df[numeric_cols].mask(df[numeric_cols] < 0)

# Erneut fehlende Werte entfernen
df = df.dropna()

In [132]:
df.index.duplicated().sum()

np.int64(0)

Es gibt keine Dublikatten

### 7. Speichern des bereinigten Datensatzes

Die bereinigte Version wird unter data/processed/ gespeichert und dient als Basis für:

- EDA
- Modellierung
- Streamlit-App
- Power BI Dashboard

In [133]:
df = df.astype('float32') #Speicheroptimierung machen

In [134]:
df.to_csv(pfad_bearbeitet)

In [135]:
df.describe() #die Verteilung prüfen

,Global_active_power,Global_reactive_power,Voltage,Global_intensity,Sub_metering_1,Sub_metering_2,Sub_metering_3
count,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06,2.049280e+06
mean,1.091615e+00,1.237145e-01,2.408398e+02,4.627758e+00,1.121923e+00,1.298520e+00,6.458447e+00
std,1.057294e+00,1.127220e-01,3.239987e+00,4.444396e+00,6.153031e+00,5.822026e+00,8.437154e+00
min,7.600000e-02,0.000000e+00,2.232000e+02,2.000000e-01,0.000000e+00,0.000000e+00,0.000000e+00
25%,3.080000e-01,4.800000e-02,2.389900e+02,1.400000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,6.020000e-01,1.000000e-01,2.410100e+02,2.600000e+00,0.000000e+00,0.000000e+00,1.000000e+00
75%,1.528000e+00,1.940000e-01,2.428900e+02,6.400000e+00,0.000000e+00,1.000000e+00,1.700000e+01
max,1.112200e+01,1.390000e+00,2.541500e+02,4.840000e+01,8.800000e+01,8.000000e+01,3.100000e+01
